In [1]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import load_model
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold

from matplotlib import font_manager, rc
#matplotlib 한글 폰트 오류 문제 해결
font_path='./dataset/malgun.ttf'
font_name = font_manager.FontProperties(fname=font_path).get_name()
rc('font',family=font_name)

## 1. 데이터 살펴보기

In [2]:
df = pd.read_csv('./dataset/sonar.csv', header=None)
df.head()

,0,1,2,3,4,5,6,7,8,9,...,51,52,53,54,55,56,57,58,59,60
0,0.0200,0.0371,0.0428,0.0207,0.0954,0.0986,0.1539,0.1601,0.3109,0.2111,...,0.0027,0.0065,0.0159,0.0072,0.0167,0.0180,0.0084,0.0090,0.0032,R
1,0.0453,0.0523,0.0843,0.0689,0.1183,0.2583,0.2156,0.3481,0.3337,0.2872,...,0.0084,0.0089,0.0048,0.0094,0.0191,0.0140,0.0049,0.0052,0.0044,R
2,0.0262,0.0582,0.1099,0.1083,0.0974,0.2280,0.2431,0.3771,0.5598,0.6194,...,0.0232,0.0166,0.0095,0.0180,0.0244,0.0316,0.0164,0.0095,0.0078,R
3,0.0100,0.0171,0.0623,0.0205,0.0205,0.0368,0.1098,0.1276,0.0598,0.1264,...,0.0121,0.0036,0.0150,0.0085,0.0073,0.0050,0.0044,0.0040,0.0117,R
4,0.0762,0.0666,0.0481,0.0394,0.0590,0.0649,0.1209,0.2467,0.3564,0.4459,...,0.0031,0.0054,0.0105,0.0110,0.0015,0.0072,0.0048,0.0107,0.0094,R


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 208 entries, 0 to 207
Data columns (total 61 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   0       208 non-null    float64
 1   1       208 non-null    float64
 2   2       208 non-null    float64
 3   3       208 non-null    float64
 4   4       208 non-null    float64
 5   5       208 non-null    float64
 6   6       208 non-null    float64
 7   7       208 non-null    float64
 8   8       208 non-null    float64
 9   9       208 non-null    float64
 10  10      208 non-null    float64
 11  11      208 non-null    float64
 12  12      208 non-null    float64
 13  13      208 non-null    float64
 14  14      208 non-null    float64
 15  15      208 non-null    float64
 16  16      208 non-null    float64
 17  17      208 non-null    float64
 18  18      208 non-null    float64
 19  19      208 non-null    float64
 20  20      208 non-null    float64
 21  21      208 non-null    float64
 22  22

In [4]:
df.describe()

,0,1,2,3,4,5,6,7,8,9,...,50,51,52,53,54,55,56,57,58,59
count,208.000000,208.000000,208.000000,208.000000,208.000000,208.000000,208.000000,208.000000,208.000000,208.000000,...,208.000000,208.000000,208.000000,208.000000,208.000000,208.000000,208.000000,208.000000,208.000000,208.000000
mean,0.029164,0.038437,0.043832,0.053892,0.075202,0.104570,0.121747,0.134799,0.178003,0.208259,...,0.016069,0.013420,0.010709,0.010941,0.009290,0.008222,0.007820,0.007949,0.007941,0.006507
std,0.022991,0.032960,0.038428,0.046528,0.055552,0.059105,0.061788,0.085152,0.118387,0.134416,...,0.012008,0.009634,0.007060,0.007301,0.007088,0.005736,0.005785,0.006470,0.006181,0.005031
min,0.001500,0.000600,0.001500,0.005800,0.006700,0.010200,0.003300,0.005500,0.007500,0.011300,...,0.000000,0.000800,0.000500,0.001000,0.000600,0.000400,0.000300,0.000300,0.000100,0.000600
25%,0.013350,0.016450,0.018950,0.024375,0.038050,0.067025,0.080900,0.080425,0.097025,0.111275,...,0.008425,0.007275,0.005075,0.005375,0.004150,0.004400,0.003700,0.003600,0.003675,0.003100
50%,0.022800,0.030800,0.034300,0.044050,0.062500,0.092150,0.106950,0.112100,0.152250,0.182400,...,0.013900,0.011400,0.009550,0.009300,0.007500,0.006850,0.005950,0.005800,0.006400,0.005300
75%,0.035550,0.047950,0.057950,0.064500,0.100275,0.134125,0.154000,0.169600,0.233425,0.268700,...,0.020825,0.016725,0.014900,0.014500,0.012100,0.010575,0.010425,0.010350,0.010325,0.008525
max,0.137100,0.233900,0.305900,0.426400,0.401000,0.382300,0.372900,0.459000,0.682800,0.710600,...,0.100400,0.070900,0.039000,0.035200,0.044700,0.039400,0.035500,0.044000,0.036400,0.043900


## 2. seed 설정

In [5]:
seed = 0
np.random.seed(seed)
tf.random.set_seed(seed)

## 3. Attribute, Class 나누기

In [6]:
dataset = df.values
X = dataset[:,0:60].astype(float)
Y_obj = dataset[:,60]

## 4. Categorical Class를 one-hot encoding 변환

In [7]:
e = LabelEncoder()
e.fit(Y_obj)
Y = e.transform(Y_obj)

## 5. K-fold Cross Validation 설정

In [8]:
n_fold = 10
skf = StratifiedKFold(n_splits=n_fold, shuffle=True, random_state=seed)

# 빈 accuracy 배열
accuracy = []

## 6. 딥러닝 모델 설정, 컴파일, 실행

In [9]:
for train, test in skf.split(X, Y):
    # 모델 설정
    model = Sequential()
    model.add(Dense(24,  input_dim=60, activation='relu'))
    model.add(Dense(10, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))

    #모델 컴파일
    model.compile(loss='mean_squared_error', optimizer='adam', metrics=['accuracy'])
    
    # 모델 실행
    model.fit(X[train], Y[train], epochs=100, batch_size=10)
    
    k_accuracy = "%.4f" % (model.evaluate(X[test], Y[test])[1])
    accuracy.append(k_accuracy)

Train on 187 samples
Epoch 1/100
187/187 [==============================] - 0s 2ms/sample - loss: 0.2423 - accuracy: 0.5668
Epoch 2/100
187/187 [==============================] - 0s 149us/sample - loss: 0.2286 - accuracy: 0.6310
Epoch 3/100
187/187 [==============================] - 0s 165us/sample - loss: 0.2187 - accuracy: 0.6631
Epoch 4/100
187/187 [==============================] - 0s 155us/sample - loss: 0.2114 - accuracy: 0.7326
Epoch 5/100
187/187 [==============================] - 0s 155us/sample - loss: 0.2078 - accuracy: 0.7166
Epoch 6/100
187/187 [==============================] - 0s 149us/sample - loss: 0.1948 - accuracy: 0.7914
Epoch 7/100
187/187 [==============================] - 0s 165us/sample - loss: 0.1877 - accuracy: 0.7968
Epoch 8/100
187/187 [==============================] - 0s 155us/sample - loss: 0.1818 - accuracy: 0.7861
Epoch 9/100
187/187 [==============================] - 0s 155us/sample - loss: 0.1747 - accuracy: 0.8021
Epoch 10/100
187/187 [==============

187/187 [==============================] - 0s 149us/sample - loss: 0.0552 - accuracy: 0.9465
Epoch 79/100
187/187 [==============================] - 0s 160us/sample - loss: 0.0518 - accuracy: 0.9465
Epoch 80/100
187/187 [==============================] - 0s 149us/sample - loss: 0.0510 - accuracy: 0.9465
Epoch 81/100
187/187 [==============================] - 0s 144us/sample - loss: 0.0508 - accuracy: 0.9465
Epoch 82/100
187/187 [==============================] - 0s 128us/sample - loss: 0.0490 - accuracy: 0.9519
Epoch 83/100
187/187 [==============================] - 0s 123us/sample - loss: 0.0519 - accuracy: 0.9358
Epoch 84/100
187/187 [==============================] - 0s 128us/sample - loss: 0.0481 - accuracy: 0.9465
Epoch 85/100
187/187 [==============================] - 0s 144us/sample - loss: 0.0478 - accuracy: 0.9519
Epoch 86/100
187/187 [==============================] - 0s 133us/sample - loss: 0.0478 - accuracy: 0.9412
Epoch 87/100
187/187 [==============================] - 0s 

187/187 [==============================] - 0s 128us/sample - loss: 0.1021 - accuracy: 0.8770
Epoch 55/100
187/187 [==============================] - 0s 139us/sample - loss: 0.0995 - accuracy: 0.8663
Epoch 56/100
187/187 [==============================] - 0s 128us/sample - loss: 0.1031 - accuracy: 0.8449
Epoch 57/100
187/187 [==============================] - 0s 139us/sample - loss: 0.1072 - accuracy: 0.8449
Epoch 58/100
187/187 [==============================] - 0s 144us/sample - loss: 0.1015 - accuracy: 0.8717
Epoch 59/100
187/187 [==============================] - 0s 128us/sample - loss: 0.0967 - accuracy: 0.8663
Epoch 60/100
187/187 [==============================] - 0s 133us/sample - loss: 0.0959 - accuracy: 0.8770
Epoch 61/100
187/187 [==============================] - 0s 133us/sample - loss: 0.0940 - accuracy: 0.8930
Epoch 62/100
187/187 [==============================] - 0s 128us/sample - loss: 0.0931 - accuracy: 0.8717
Epoch 63/100
187/187 [==============================] - 0s 

187/187 [==============================] - 0s 208us/sample - loss: 0.1215 - accuracy: 0.8663
Epoch 31/100
187/187 [==============================] - 0s 171us/sample - loss: 0.1176 - accuracy: 0.8770
Epoch 32/100
187/187 [==============================] - 0s 176us/sample - loss: 0.1168 - accuracy: 0.8770
Epoch 33/100
187/187 [==============================] - 0s 133us/sample - loss: 0.1140 - accuracy: 0.8877
Epoch 34/100
187/187 [==============================] - 0s 123us/sample - loss: 0.1137 - accuracy: 0.8770
Epoch 35/100
187/187 [==============================] - 0s 139us/sample - loss: 0.1129 - accuracy: 0.8610
Epoch 36/100
187/187 [==============================] - 0s 128us/sample - loss: 0.1168 - accuracy: 0.8449
Epoch 37/100
187/187 [==============================] - 0s 133us/sample - loss: 0.1097 - accuracy: 0.8824
Epoch 38/100
187/187 [==============================] - 0s 128us/sample - loss: 0.1054 - accuracy: 0.8877
Epoch 39/100
187/187 [==============================] - 0s 

187/187 [==============================] - 0s 128us/sample - loss: 0.2215 - accuracy: 0.7326
Epoch 7/100
187/187 [==============================] - 0s 128us/sample - loss: 0.2161 - accuracy: 0.7273
Epoch 8/100
187/187 [==============================] - 0s 128us/sample - loss: 0.2098 - accuracy: 0.7326
Epoch 9/100
187/187 [==============================] - 0s 128us/sample - loss: 0.2022 - accuracy: 0.7754
Epoch 10/100
187/187 [==============================] - 0s 155us/sample - loss: 0.1964 - accuracy: 0.7594
Epoch 11/100
187/187 [==============================] - 0s 128us/sample - loss: 0.1895 - accuracy: 0.7807
Epoch 12/100
187/187 [==============================] - 0s 139us/sample - loss: 0.1828 - accuracy: 0.7807
Epoch 13/100
187/187 [==============================] - 0s 235us/sample - loss: 0.1764 - accuracy: 0.7807
Epoch 14/100
187/187 [==============================] - 0s 197us/sample - loss: 0.1697 - accuracy: 0.8342
Epoch 15/100
187/187 [==============================] - 0s 133

...

187/187 [==============================] - 0s 219us/sample - loss: 0.0318 - accuracy: 0.9786
Epoch 89/100
187/187 [==============================] - 0s 224us/sample - loss: 0.0301 - accuracy: 0.9786
Epoch 90/100
187/187 [==============================] - 0s 203us/sample - loss: 0.0280 - accuracy: 0.9947
Epoch 91/100
187/187 [==============================] - 0s 208us/sample - loss: 0.0264 - accuracy: 0.9893
Epoch 92/100
187/187 [==============================] - 0s 197us/sample - loss: 0.0266 - accuracy: 0.9947
Epoch 93/100
187/187 [==============================] - 0s 197us/sample - loss: 0.0246 - accuracy: 0.9947
Epoch 94/100
187/187 [==============================] - 0s 197us/sample - loss: 0.0239 - accuracy: 0.9947
Epoch 95/100
187/187 [==============================] - 0s 197us/sample - loss: 0.0241 - accuracy: 0.9893
Epoch 96/100
187/187 [==============================] - 0s 203us/sample - loss: 0.0238 - accuracy: 0.9947
Epoch 97/100
187/187 [==============================] - 0s 

187/187 [==============================] - 0s 229us/sample - loss: 0.0625 - accuracy: 0.9412
Epoch 65/100
187/187 [==============================] - 0s 203us/sample - loss: 0.0707 - accuracy: 0.9144
Epoch 66/100
187/187 [==============================] - 0s 213us/sample - loss: 0.0671 - accuracy: 0.9251
Epoch 67/100
187/187 [==============================] - 0s 208us/sample - loss: 0.0624 - accuracy: 0.9251
Epoch 68/100
187/187 [==============================] - 0s 192us/sample - loss: 0.0603 - accuracy: 0.9412
Epoch 69/100
187/187 [==============================] - 0s 192us/sample - loss: 0.0589 - accuracy: 0.9626
Epoch 70/100
187/187 [==============================] - 0s 192us/sample - loss: 0.0586 - accuracy: 0.9412
Epoch 71/100
187/187 [==============================] - 0s 192us/sample - loss: 0.0571 - accuracy: 0.9519
Epoch 72/100
187/187 [==============================] - 0s 187us/sample - loss: 0.0563 - accuracy: 0.9412
Epoch 73/100
187/187 [==============================] - 0s 

188/188 [==============================] - 0s 138us/sample - loss: 0.1140 - accuracy: 0.8723
Epoch 41/100
188/188 [==============================] - 0s 143us/sample - loss: 0.1152 - accuracy: 0.8617
Epoch 42/100
188/188 [==============================] - 0s 127us/sample - loss: 0.1107 - accuracy: 0.8723
Epoch 43/100
188/188 [==============================] - 0s 133us/sample - loss: 0.1086 - accuracy: 0.8670
Epoch 44/100
188/188 [==============================] - 0s 143us/sample - loss: 0.1097 - accuracy: 0.8617
Epoch 45/100
188/188 [==============================] - 0s 127us/sample - loss: 0.1067 - accuracy: 0.8830
Epoch 46/100
188/188 [==============================] - 0s 138us/sample - loss: 0.1112 - accuracy: 0.8617
Epoch 47/100
188/188 [==============================] - 0s 133us/sample - loss: 0.1047 - accuracy: 0.8989
Epoch 48/100
188/188 [==============================] - 0s 127us/sample - loss: 0.1036 - accuracy: 0.8723
Epoch 49/100
188/188 [==============================] - 0s 

188/188 [==============================] - 0s 138us/sample - loss: 0.1774 - accuracy: 0.7660
Epoch 17/100
188/188 [==============================] - 0s 154us/sample - loss: 0.1753 - accuracy: 0.7819
Epoch 18/100
188/188 [==============================] - 0s 127us/sample - loss: 0.1689 - accuracy: 0.7766
Epoch 19/100
188/188 [==============================] - 0s 133us/sample - loss: 0.1621 - accuracy: 0.8032
Epoch 20/100
188/188 [==============================] - 0s 138us/sample - loss: 0.1593 - accuracy: 0.7926
Epoch 21/100
188/188 [==============================] - 0s 127us/sample - loss: 0.1542 - accuracy: 0.8191
Epoch 22/100
188/188 [==============================] - 0s 138us/sample - loss: 0.1525 - accuracy: 0.7979
Epoch 23/100
188/188 [==============================] - 0s 127us/sample - loss: 0.1506 - accuracy: 0.7713
Epoch 24/100
188/188 [==============================] - 0s 138us/sample - loss: 0.1443 - accuracy: 0.8298
Epoch 25/100
188/188 [==============================] - 0s 

188/188 [==============================] - 0s 228us/sample - loss: 0.0650 - accuracy: 0.9468
Epoch 94/100
188/188 [==============================] - 0s 180us/sample - loss: 0.0638 - accuracy: 0.9415
Epoch 95/100
188/188 [==============================] - 0s 164us/sample - loss: 0.0638 - accuracy: 0.9415
Epoch 96/100
188/188 [==============================] - 0s 159us/sample - loss: 0.0670 - accuracy: 0.9362
Epoch 97/100
188/188 [==============================] - 0s 159us/sample - loss: 0.0660 - accuracy: 0.9362
Epoch 98/100
188/188 [==============================] - 0s 159us/sample - loss: 0.0627 - accuracy: 0.9362
Epoch 99/100
188/188 [==============================] - 0s 164us/sample - loss: 0.0592 - accuracy: 0.9415
Epoch 100/100
20/20 [==============================] - 0s 4ms/sample - loss: 0.1362 - accuracy: 0.8500


## 8. 모델 저장 및 재호출

In [10]:
model.save("my_model.h5")

del model
model = load_model('my_model.h5')

## 9.  모델 정확도 검증

In [11]:
print("%.f fold Accuray :" % n_fold, accuracy)

10 fold Accuray : ['0.7143', '0.9048', '0.8095', '0.9048', '0.7619', '0.8095', '0.8095', '0.7619', '0.9500', '0.8500']
